In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [9]:
api_wrapper =WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [10]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents =RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3.2:1b"))
retreiver = vectordb.as_retriever()
retreiver

C:\Users\Kartikeya\AppData\Local\Temp\ipykernel_19940\91144827.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3.2:1b"))


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FCFFC93550>, search_kwargs={})

In [6]:
from langchain.tools.retriever import create_retriever_tool
retreiver_tool = create_retriever_tool(retreiver,"langsmith_search",'Search for information about langsmith.For any question about langsmith')

In [7]:
# Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools = [wiki,arxiv,retreiver_tool]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\gen-ai\\langcain-tut\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about langsmith.For any question about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FCFB2991C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS objec

In [13]:
from langchain_community.llms.ollama import Ollama
# Load Ollama llama3.2:1b llm model
llm = Ollama(model ='llama3.2:1b')
llm

C:\Users\Kartikeya\AppData\Local\Temp\ipykernel_19940\3172814806.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model ='llama3.2:1b')


Ollama(model='llama3.2:1b')

In [62]:
from langchain.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]

Question: {input}
Thought: {agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

In [65]:
from langchain.agents import create_react_agent
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

In [66]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    output_keys=["output"],
    handle_parsing_errors=True 
)

In [ ]:
result = agent_executor.invoke({"input": "tell me about langsmith"})
result



> Entering new AgentExecutor chain...
Question: Tell me about langsmith
Thought: I need to search for information on langsmith using a suitable tool.Invalid Format: Missing 'Action:' after 'Thought:'Question: Tell me about langsmith
Thought: I need to search for information on langsmith using a suitable tool.Invalid Format: Missing 'Action:' after 'Thought:'Question: Tell me about langsmith
Thought: I need to search for information on langsmith using a suitable tool.Invalid Format: Missing 'Action:' after 'Thought:'Question: Tell me about langsmith
I need to search for information on langsmith using a suitable tool.Invalid Format: Missing 'Action:' after 'Thought:'Question: I need to search for information on langsmith using a suitable tool.
Action: Use the arxiv, wikipedia, or langsmith_search tools.Invalid Format: Missing 'Action Input:' after 'Action:'I can assist you with your questions. Here are the answers based on the provided instructions:

Question: What is langsmith?
Though

{'input': 'tell me about langsmith',
 'output': 'Agent stopped due to iteration limit or time limit.'}